$$
E_{06} = 1\cdot \tilde P_{06} + \sum_{k=1}^{5} (1+E_{k6})\cdot \tilde{P}_{0k} \\
\vdots \\
E_{i6} = 1\cdot \tilde P_{i6} + \sum_{\substack{k=0 \\ k\neq i}}^{5} (1+E_{k6})\cdot \tilde{P}_{ik} \\
\vdots \\
E_{56} = 1\cdot \tilde P_{56} + \sum_{k=0}^{4} (1+E_{k6})\cdot \tilde{P}_{5k} \\
$$

$$
E_{66} = \sum_{k=0}^{5} (1+E_{k6})\cdot \tilde{P}_{6k}
$$

$$
E^*_{67} = 1\cdot P_{67} + (E_{66} + E^*_{67}) \cdot (1 - P_{67}) \\
E^*_{67} = 1\cdot P_{67} + E_{66} \cdot (1 - P_{67}) + E^*_{67} \cdot (1 - P_{67}) \\
P_{67} \cdot E^*_{67} = P_{67} + E_{66} \cdot (1 - P_{67}) \\
E^*_{67} = \frac{P_{67} + E_{66} \cdot (1 - P_{67})}{P_{67}}
$$

$$
E^*_{07} = E_{06} + E^*_{67}
$$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import operator as op
from functools import reduce

def comb(n, k):
    k = min(k, n-k)
    num = reduce(op.mul, range(n, n-k, -1), 1)
    den = reduce(op.mul, range(1, k+1), 1)
    return num // den

In [3]:
def bit_diff(a, b):
    x, c = a ^ b, 0
    while x > 0:
        c += 1
        x -= x & -x
    return c

In [4]:
def get_dist(u, v):
    return np.sqrt(bit_diff(u, v))

In [5]:
def get_prob_const(dim):
    return 1 / sum(1/np.sqrt(i) * comb(dim, i) for i in range(1, dim+1))

In [6]:
def get_prob(u, v, C):
    if u == v: return 0
    return C / get_dist(u, v)

In [7]:
def get_prob_matrix(n_vertices, C):
    return np.array([[get_prob(u, v, C) for v in range(n_vertices)] for u in range(n_vertices)])

In [8]:
def update_E(E, P2, n_vertices):
    for u in range(n_vertices-2):
        E[u] = P2[u, n_vertices-2]
        for v in range(n_vertices-2):
            if u == v: continue
            E[u] += (1 + E[v]) * P2[u, v]
    return E

In [9]:
def run_epochs(E, n_iters, P2, n_vertices, Es=None):
    for _ in range(n_iters):
        E = update_E(E, P2, n_vertices)
        if Es: Es.append(E[0])
    return E

In [10]:
def get_E66(E, P2, n_vertices):
    return sum((1 + E[i]) * P2[n_vertices-2, i] for i in range(n_vertices-2))

In [11]:
def solve(dim, n_iter=3000):
    n_vertices = 2**dim
    C = get_prob_const(dim)
    P = get_prob_matrix(n_vertices, C)
    P2 = P.copy()
    P2[:, -1] = 0
    P2 = P2 / P2.sum(axis=1, keepdims=True)
    E, Es = np.zeros(n_vertices-2), [0]
    E = run_epochs(E, n_iter, P2, n_vertices, Es)
    E66 = get_E66(E, P2, n_vertices)
    p67 = P[n_vertices-2, n_vertices-1]
    return (p67 + E66*(1-p67))/p67 + E[0]

In [12]:
E3 = solve(3)
E3

41.606568606071235

In [13]:
E7 = solve(7)
E7

9223.28200147903